In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
import os
from tensorflow.examples.tutorials.mnist import input_data
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train = pd.read_csv(r'C:/Users/12239/Desktop/CFF/离散制造过程中典型工件的质量符合率预测/train_set/first_round_training_data - 副本.csv')
test = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\test_set\first_round_testing_data - 副本.csv')
submit = pd.read_csv(r'C:\Users\12239\Desktop\CFF\离散制造过程中典型工件的质量符合率预测\submit_example.csv')

In [12]:
sess = tf.InteractiveSession()

In [3]:
feature_name = ['Parameter{0}'.format(i) for i in range(5,11)]
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
train['label'] = train['Quality_label'].map(dit)
train = pd.get_dummies(train, columns=['Quality_label'])
bin_label = ['Quality_label_Excellent', 'Quality_label_Good', 'Quality_label_Pass', 'Quality_label_Fail']

In [16]:
train_x = train[feature_name].values
train_y = train[bin_label].values
x_test = test[feature_name].values

In [6]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_1x1(x):
    return tf.nn.max_pool(x,ksize=[1,1,1,1],strides=[1,1,1,1],padding='SAME')
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [9]:
n_input = 6
n_classes = 4

x = tf.placeholder("float",[None,n_input])
y = tf.placeholder("float",[None,n_classes])
x_image=tf.reshape(x,[-1,2,3,1])

w_conv1 = weight_variable([2,1,1,3])
b_cinv1 = bias_variable([3])
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1) + b_cinv1)
h_pool1 = max_pool_1x1(h_conv1)

w_conv2 = weight_variable([1,1,1,6])
b_cinv2 = bias_variable([6])
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2) + b_cinv2)
h_pool2 = max_pool_1x1(h_conv2)

w_fc1 = weight_variable([6*2*3,12])
b_fc1 = bias_variable([12])
h_pool2_flat = tf.reshape(h_pool2,[-1,6*2*3])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
w_fc2 = weight_variable([12,4])
b_fc2 = bias_variable([4])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_conv),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.math.argmax(y_conv,1),tf.arg_max(y,1))
accuracy  =tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [19]:
tf.global_variables_initializer().run()
for i in range(60):
    train_accuarcy = accuracy.eval(feed_dict={x:train_x[100*i:100*i+99,:],y:train_y[100*i:100*i+99,:],keep_prob:1.0})
    print("step %d,train_accuarcy %g"%(i,train_accuarcy))
    train_step.run(feed_dict={x:train_x[100*i:100*i+99,:],y:train_y[100*i:100*i+99,:],keep_prob:0.5})

step 0,train_accuarcy 0.30303
step 1,train_accuarcy 0.191919
step 2,train_accuarcy 0.161616
step 3,train_accuarcy 0.141414
step 4,train_accuarcy 0.171717
step 5,train_accuarcy 0.191919
step 6,train_accuarcy 0.222222
step 7,train_accuarcy 0.131313
step 8,train_accuarcy 0.363636
step 9,train_accuarcy 0.141414
step 10,train_accuarcy 0.121212
step 11,train_accuarcy 0.20202
step 12,train_accuarcy 0.131313
step 13,train_accuarcy 0.050505
step 14,train_accuarcy 0.171717
step 15,train_accuarcy 0.161616
step 16,train_accuarcy 0.131313
step 17,train_accuarcy 0.323232
step 18,train_accuarcy 0.191919
step 19,train_accuarcy 0.171717
step 20,train_accuarcy 0.10101
step 21,train_accuarcy 0.434343
step 22,train_accuarcy 0.181818
step 23,train_accuarcy 0.191919
step 24,train_accuarcy 0.0909091
step 25,train_accuarcy 0.171717
step 26,train_accuarcy 0.20202
step 27,train_accuarcy 0.161616
step 28,train_accuarcy 0.242424
step 29,train_accuarcy 0.282828
step 30,train_accuarcy 0.20202
step 31,train_accuarcy